# Decision Tree classification


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.chdir("..")

In [2]:
from IPython.core.debugger import Tracer
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from scipy.signal import welch
from IPython.core.debugger import Tracer
from sklearn_pandas import DataFrameMapper, cross_val_score
from p300.feature_extraction import BaseTransformer

In [3]:
hdf = pd.HDFStore("output/instances.h5")


hdf.keys()

['/subjects/s10229001',
 '/subjects/s10444001',
 '/subjects/s10729001',
 '/subjects/s10882001',
 '/subjects/s10924001',
 '/subjects/s11551001',
 '/subjects/s11627001',
 '/subjects/s11632001',
 '/subjects/s11693001',
 '/subjects/s12137001',
 '/subjects/s12168001',
 '/subjects/s12521001',
 '/subjects/s12702001',
 '/subjects/s12900001',
 '/subjects/s13235001',
 '/subjects/s13252001',
 '/subjects/s13431001',
 '/subjects/s13640002',
 '/subjects/s13863001',
 '/subjects/s14023001',
 '/subjects/s1414001',
 '/subjects/s1491001',
 '/subjects/s14998001',
 '/subjects/s15362001',
 '/subjects/s15424001',
 '/subjects/s15641001',
 '/subjects/s16003001',
 '/subjects/s1609001',
 '/subjects/s16266001',
 '/subjects/s1635001',
 '/subjects/s16637001',
 '/subjects/s16683001',
 '/subjects/s16779001',
 '/subjects/s16893001',
 '/subjects/s16943001',
 '/subjects/s17005001',
 '/subjects/s17435001',
 '/subjects/s17436001',
 '/subjects/s17576001',
 '/subjects/s17674001',
 '/subjects/s17962001',
 '/subjects/s1804600

Veamos cuántos sujetos tenemos

Tomemos uno: 29164001

In [5]:
from sklearn.model_selection import cross_val_score

non_features = ['id', 'array_path', 'ch_names', 'event_time', 'event_type', 'index',
       'sfreq', 'subject_id', 'target']

def get_data_from_subject(hdf, key):
    df = hdf[key]
    X = df[df.columns.difference(non_features)].as_matrix()
    y = df.target.as_matrix()
    
    return X, y

X, y = get_data_from_subject(hdf, '/subjects/s5224001')

X.shape

(1980, 112)

# SVM

In [14]:
from sklearn.svm import LinearSVC

clf = LinearSVC()

In [15]:
cross_val_score(clf, X, y, scoring='roc_auc', cv=10).mean()

0.52389348025711668

# LDA

In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()

In [19]:
cross_val_score(clf, X, y, scoring='roc_auc', cv=10).mean()

0.7092194674012855

# Balance de las clases

In [23]:
total = len(y)
no_targets = sum(y)
print("Total samples: {} - Total positives {}".format(total, no_targets))
print("Ratio : {}".format(no_targets / total))

Total samples: 1980 - Total positives 330
Ratio : 0.16666666666666666


Como vemos, el balance es 1 target cada 6. Eso es porque el P300 tiene 6 columnas y filas.

# LDA con todos los sujetos

In [24]:
results = []
clf = LinearDiscriminantAnalysis()

for key in hdf.keys():
    print("Analyzing {}".format(key))
    clf = LinearDiscriminantAnalysis()
    
    X, y = get_data_from_subject(hdf, key)
    auc = cross_val_score(clf, X, y, scoring='roc_auc').mean()
    precision = cross_val_score(clf, X, y, scoring='precision').mean()
    recall = cross_val_score(clf, X, y, scoring='recall').mean()
    accuracy = cross_val_score(clf, X, y, scoring='accuracy').mean()
    
    results.append({
        'subject_id': key,
        'auc': auc,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    })
    


Analyzing /subjects/s10229001
Analyzing /subjects/s10444001
Analyzing /subjects/s10729001
Analyzing /subjects/s10882001
Analyzing /subjects/s10924001
Analyzing /subjects/s11551001
Analyzing /subjects/s11627001
Analyzing /subjects/s11632001
Analyzing /subjects/s11693001
Analyzing /subjects/s12137001
Analyzing /subjects/s12168001
Analyzing /subjects/s12521001
Analyzing /subjects/s12702001
Analyzing /subjects/s12900001
Analyzing /subjects/s13235001
Analyzing /subjects/s13252001
Analyzing /subjects/s13431001
Analyzing /subjects/s13640002
Analyzing /subjects/s13863001
Analyzing /subjects/s14023001
Analyzing /subjects/s1414001
Analyzing /subjects/s1491001
Analyzing /subjects/s14998001
Analyzing /subjects/s15362001
Analyzing /subjects/s15424001
Analyzing /subjects/s15641001
Analyzing /subjects/s16003001
Analyzing /subjects/s1609001
Analyzing /subjects/s16266001
Analyzing /subjects/s1635001
Analyzing /subjects/s16637001
Analyzing /subjects/s16683001
Analyzing /subjects/s16779001
Analyzing /sub

In [25]:
pd.set_option('display.max_rows', 500)

results_df = pd.DataFrame(results)

results_df

,accuracy,auc,precision,recall,subject_id
0,0.802222,0.643967,0.374225,0.226667,/subjects/s10229001
1,0.811616,0.668518,0.348270,0.133333,/subjects/s10444001
2,0.822593,0.638785,0.404832,0.091111,/subjects/s10729001
3,0.824747,0.738496,0.465224,0.254545,/subjects/s10882001
4,0.824206,0.678772,0.428338,0.197619,/subjects/s10924001
5,0.823333,0.664313,0.450487,0.200000,/subjects/s11551001
6,0.812778,0.592347,0.210445,0.050000,/subjects/s11627001
7,0.838889,0.770187,0.558276,0.278788,/subjects/s11632001
8,0.813636,0.656242,0.378002,0.169697,/subjects/s11693001
9,0.822619,0.718673,0.438704,0.228571,/subjects/s12137001


In [26]:
results_df.mean()

accuracy     0.819373
auc          0.654100
precision    0.360827
recall       0.169186
dtype: float64